**Install required libraries**

In [4]:
!pip install tensorflow keras
!pip install facenet-pytorch
!pip install mtcnn # MTCNN is used for face detection

**Mount Google Drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [6]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow.keras.applications import InceptionResNetV2
from mtcnn.mtcnn import MTCNN


 Set Up Paths and Parameters:

In [7]:
# Path to your dataset
dataset_path = '/content/drive/MyDrive/EIT/FaceDetection faceNet/Dataset'

# Initialize face detector
detector = MTCNN()


**Face Extraction Function**

In [9]:
def extract_face(image, required_size=(160, 160)):
    # Detect faces in the image
    faces = detector.detect_faces(image)
    if len(faces) == 0:
        return None
    # Extract the face
    x1, y1, width, height = faces[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    # Resize the face to required size
    face = cv2.resize(face, required_size)
    return face


**Load and Preprocess the Dataset:**

In [11]:
def load_dataset(directory):
    X, y = [], []
    for person_name in os.listdir(directory):
        person_path = os.path.join(directory, person_name)
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            image = cv2.imread(image_path)
            face = extract_face(image)
            if face is not None:
                X.append(face)
                y.append(person_name)
    return np.array(X), np.array(y)

# Load dataset
X, y = load_dataset(dataset_path)

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Save the label encoder
import joblib
joblib.dump(encoder, '/content/drive/MyDrive/EIT/FaceDetection faceNet/Model/label_encoder.pkl')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 753ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━

['/content/drive/MyDrive/EIT/FaceDetection faceNet/Model/label_encoder.pkl']

**Load FaceNet Model**

In [12]:
from tensorflow.keras.applications import InceptionResNetV2

# Load FaceNet model with ImageNet weights
facenet_model = InceptionResNetV2(input_shape=(160, 160, 3), weights='imagenet', include_top=False, pooling='avg')

# Generate embeddings for the dataset
def get_embeddings(model, X):
    embeddings = []
    for face in X:
        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        sample = np.expand_dims(face, axis=0)
        embedding = model.predict(sample)
        embeddings.append(embedding[0])
    return np.array(embeddings)

# Generate embeddings for training and testing sets
X_train_embeddings = get_embeddings(facenet_model, X_train)
X_test_embeddings = get_embeddings(facenet_model, X_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

**Step 5: Train a Classifier**

**Train an SVM Classifier**

In [13]:
# Train an SVM classifier on the embeddings
model = SVC(kernel='linear', probability=True)
model.fit(X_train_embeddings, y_train)

# Test the classifier
accuracy = model.score(X_test_embeddings, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 95.65%


**Test the Model with New Images**

In [14]:
def predict(model, facenet_model, image_path):
    image = cv2.imread(image_path)
    face = extract_face(image)
    if face is not None:
        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        sample = np.expand_dims(face, axis=0)
        embedding = facenet_model.predict(sample)
        prediction = model.predict(embedding)
        prob = model.predict_proba(embedding)
        return encoder.inverse_transform(prediction)[0], prob[0][prediction][0]
    else:
        return None, None

# Predict a new image
image_path = '/content/drive/MyDrive/EIT/FaceDetection faceNet/jh.jpeg'
person_name, confidence = predict(model, facenet_model, image_path)
print(f"Predicted: {person_name} with confidence: {confidence}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Predicted: Jenny with confidence: 0.8657036759936458


**Save the Model**

In [15]:
# Save the trained classifier model
import joblib
joblib.dump(model, '/content/drive/MyDrive/EIT/FaceDetection faceNet/Model/face_recognition_svm.pkl')


['/content/drive/MyDrive/EIT/FaceDetection faceNet/Model/face_recognition_svm.pkl']

**Test with code**

In [18]:
import joblib
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from keras.applications import InceptionResNetV2
from mtcnn.mtcnn import MTCNN

# Load the FaceNet model and the trained SVM model
facenet_model = InceptionResNetV2(input_shape=(160, 160, 3), weights='imagenet', include_top=False, pooling='avg')
model = joblib.load('/content/drive/MyDrive/EIT/FaceDetection faceNet/Model/face_recognition_svm.pkl')

# Load the label encoder
encoder = joblib.load('/content/drive/MyDrive/EIT/FaceDetection faceNet/Model/label_encoder.pkl')

# Function to extract face and predict identity
def predict(model, facenet_model, image_path):
    image = cv2.imread(image_path)
    detector = MTCNN()
    results = detector.detect_faces(image)

    if results:
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = image[y1:y2, x1:x2]
        face = cv2.resize(face, (160, 160))

        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        sample = np.expand_dims(face, axis=0)
        embedding = facenet_model.predict(sample)
        prediction = model.predict(embedding)
        prob = model.predict_proba(embedding)
        return encoder.inverse_transform(prediction)[0], prob[0][prediction][0]
    else:
        return None, None


In [19]:
# Predict a new image
image_path = '/content/drive/MyDrive/EIT/FaceDetection faceNet/jh.jpeg'
person_name, confidence = predict(model, facenet_model, image_path)
print(f"Predicted: {person_name} with confidence: {confidence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Predicted: Jenny with confidence: 0.8657036759936458
